In [ ]:
import asyncio
import websockets
import json
import datetime

async def binance_ws(symbol):
    """Fetch trade data from Binance WebSocket and process OHLC."""
    global intervals, next_interval_start_ts

    # Construct the WebSocket URL dynamically based on the symbol
    BINANCE_WS_URL = f"wss://fstream.binance.com/ws/{symbol.lower()}@trade"

    intervals = {}  # Stores trades by interval start time (milliseconds)
    next_interval_start_ts = 0  # Timestamp (ms) of the first interval to process

    # Determine the next 15s interval start time based on current UTC time
    now = datetime.datetime.utcnow()
    current_second = now.second
    next_interval_second = ((current_second // 15) + 1) * 15
    if next_interval_second >= 60:
        next_interval_start = now.replace(second=0, microsecond=0) + datetime.timedelta(minutes=1)
    else:
        next_interval_start = now.replace(second=next_interval_second, microsecond=0)
    next_interval_start_ts = int(next_interval_start.timestamp() * 1000)
    print(f"Starting script for {symbol.upper()}. First interval starts at {next_interval_start.strftime('%H:%M:%S UTC')}")

    async with websockets.connect(BINANCE_WS_URL) as websocket:
        async for message in websocket:
            data = json.loads(message)
            ts = data["T"]  # Trade timestamp in milliseconds

            # Skip trades before the initial interval
            if ts < next_interval_start_ts:
                continue

            price = float(data['p'])
            quantity = float(data['q'])

            # Calculate the 15s interval this trade belongs to
            interval_start = (ts // 15000) * 15000
            if interval_start not in intervals:
                intervals[interval_start] = []
            intervals[interval_start].append({
                'timestamp': ts,
                'price': price,
                'quantity': quantity
            })

            # Check which intervals can be processed (all intervals ending before this trade's timestamp)
            to_process = []
            for key in list(intervals.keys()):
                interval_end = key + 15000
                if interval_end <= ts:
                    to_process.append(key)

            # Process each eligible interval in order
            for key in sorted(to_process):
                trades = intervals.pop(key)
                if not trades:
                    continue

                # Compute OHLC values
                open_price = trades[0]['price']
                close_price = trades[-1]['price']
                high_price = max(trade['price'] for trade in trades)
                low_price = min(trade['price'] for trade in trades)

                # Format interval start time
                interval_start_dt = datetime.datetime.utcfromtimestamp(key / 1000)
                ohlc_time = interval_start_dt.strftime("%H:%M:%S UTC")

                # Print OHLC
                print(f"\n📊 OHLC for {symbol.upper()} at {ohlc_time}:")
                print(f"Open: {open_price:.5f}, High: {high_price:.5f}, Low: {low_price:.5f}, Close: {close_price:.5f}")

async def main(symbol):
    """Run the WebSocket connection for the specified symbol."""
    await binance_ws(symbol)

# For Jupyter Notebook
symbol = "seiusdt"  # Change this to the desired trading pair (e.g., "ethusdt", "bnbusdt", etc.)
await main(symbol)

# For .py script, uncomment below:
# if __name__ == "__main__":
#     symbol = "btcusdt"  # Change this to the desired trading pair (e.g., "ethusdt", "bnbusdt", etc.)
#     asyncio.run(main(symbol))

In [30]:
import requests
import hashlib
import urllib.parse
import hmac
import time
import logging

class BinanceAPI:
    def __init__(self, api_key: str, api_secret: str, testnet: bool = False):
        self.api_key = api_key  
        self.api_secret = api_secret
        if testnet:
            self.base_url = "https://testnet.binancefuture.com"
        else:
            self.base_url = "https://fapi.binance.com"

    def _generate_signature(self, params: dict) -> str:
        """Generate HMAC SHA256 signature for request"""
        query_string = urllib.parse.urlencode(params)
        return hmac.new(
            self.api_secret.encode("utf-8"),
            query_string.encode("utf-8"),
            hashlib.sha256,
        ).hexdigest()

    def _send_request(
    self, method: str, endpoint: str, params: dict = None, signed: bool = False
) -> dict:
        """Send request to Binance FAPI"""
        url = f"{self.base_url}{endpoint}"
        headers = {"X-MBX-APIKEY": self.api_key}

        if params is None:
            params = {}

        if signed:
            params["timestamp"] = int(time.time() * 1000)
            params["signature"] = self._generate_signature(params)

        try:
            if method == "GET":
                response = requests.get(url, headers=headers, params=params)
            elif method == "POST":
                response = requests.post(url, headers=headers, params=params)
            elif method == "DELETE":
                response = requests.delete(url, headers=headers, params=params)
            else:
                raise ValueError(f"Invalid method: {method}")

            response.raise_for_status()
            return response.json()

        except requests.exceptions.RequestException as e:
            logging.error(f"API request failed: {e}")
            logging.error(f"Response: {e.response.text}")  # Log the response for debugging
            return None

    def get_mark_price(self, symbol: str) -> dict:
        """Get mark price for symbol"""
        params = {"symbol": symbol}
        return self._send_request("GET", "/fapi/v1/premiumIndex", params)

    def get_position_risk(self, symbol: str = None) -> dict:
        """Get position information"""
        params = {}
        if symbol:
            params["symbol"] = symbol
        return self._send_request("GET", "/fapi/v3/positionRisk", params, signed=True)

    def get_symbol_info(self, symbol: str) -> dict:
        """Fetch symbol information (e.g., tick size, lot size)."""
        endpoint = "/fapi/v1/exchangeInfo"
        response = self._send_request("GET", endpoint)
        if response and "symbols" in response:
            for s in response["symbols"]:
                if s["symbol"] == symbol:
                    return s
        return None

    def create_order(
        self,
        symbol: str,
        side: str,
        order_type: str,
        quantity: float = None,
        price: float = None,
        time_in_force: str = None,
    ) -> dict:
        """
        Create a new order on Binance Futures (only for MARKET and LIMIT orders).

        Parameters:
            symbol (str): The trading pair (e.g., "BTCUSDT").
            side (str): "BUY" or "SELL".
            order_type (str): Order type ("MARKET" or "LIMIT").
            quantity (float): Order quantity.
            price (float): Order price (required for LIMIT orders).
            time_in_force (str): Time in force (required for LIMIT orders, e.g., "GTC").

        Returns:
            dict: Response from Binance API.
        """
        # Validate order type
        if order_type.upper() not in ["MARKET", "LIMIT"]:
            raise ValueError("Order type must be 'MARKET' or 'LIMIT'.")

        # Prepare parameters
        params = {
            "symbol": symbol,
            "side": side,
            "type": order_type.upper(),
        }

        # Add quantity (mandatory for both MARKET and LIMIT orders)
        if quantity is None:
            raise ValueError("Quantity is required for MARKET and LIMIT orders.")
        params["quantity"] = quantity

        # Add price and timeInForce for LIMIT orders
        if order_type.upper() == "LIMIT":
            if price is None:
                raise ValueError("Price is required for LIMIT orders.")
            if time_in_force is None:
                raise ValueError("timeInForce is required for LIMIT orders.")
            params["price"] = price
            params["timeInForce"] = time_in_force.upper()

        # Send request
        return self._send_request("POST", "/fapi/v1/order", params, signed=True)

    def ping(self) -> dict:
        """Ping the Binance API to check connectivity."""
        return self._send_request("GET", "/fapi/v1/ping")

    def get_server_time(self) -> dict:
        """Get Binance server time to confirm connectivity."""
        return self._send_request("GET", "/fapi/v1/time")

    def check_balance(self):
        endpoint = "/fapi/v2/balance"
        params = {
            "timestamp": int(time.time() * 1000)
        }
        params["signature"] = self._generate_signature(params)
        return self._send_request("GET", endpoint, params=params)



In [31]:
# Initialize connection to Binancee
binance_api = BinanceAPI(api_key="TGZ6PvNeQc3c3ctlzm0UOdkgr1fi5oEMMPXDK9Dns51VXGKYGIirlOJ8de5TYNRC", api_secret="Ng4YmUDDzq7W9l5F08qcY3Qq2OXms4xE7A9nlslDIxP2agjVWqmZbOOxCRTZEHOl", testnet=False)

# Check connectivity with ping
ping_response = binance_api.ping()
print("Ping Response:", ping_response)  # Should return an empty dictionary if successful

# Check server time
server_time = binance_api.get_server_time()
print("Server Time:", server_time)  # Should return the server time in milliseconds

# Get balance information
balance = binance_api.check_balance()
usdt_balance = next((float(item['availableBalance']) for item in balance if item['asset'] == 'USDT'), None)
print(f'USDT Available Balance: {usdt_balance}')

Ping Response: {}
Server Time: {'serverTime': 1741152603783}
USDT Available Balance: 10063.21019165


In [35]:
try:
    order_response = binance_api.create_order(
        symbol="SEIUSDT",
        side="BUY",
        order_type="LIMIT",
        quantity=30,  # Mandatory for LIMIT orders
        price=0.219,     # Mandatory for LIMIT orders
        time_in_force="GTC",  # Mandatory for LIMIT orders
    )
    print("Limit Order placed successfully:", order_response)
except ValueError as e:
    print("Error:", e)
except Exception as e:
    print("API request failed:", e)

Limit Order placed successfully: {'orderId': 6148559016, 'symbol': 'SEIUSDT', 'status': 'NEW', 'clientOrderId': '66bdwoFHJpgfM6s39wXIyA', 'price': '0.2190000', 'avgPrice': '0.00', 'origQty': '30', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.0000000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.0000000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1741152749495}


In [ ]:
import asyncio
import websockets
import json
import datetime

async def binance_ws(symbol, occ_threshold, ohc_threshold, olc_threshold, n):
    """Fetch trade data with position duration management."""
    global intervals, next_interval_start_ts, current_max_end
    
    BINANCE_WS_URL = f"wss://fstream.binance.com/ws/{symbol.lower()}@trade"
    intervals = {}
    current_max_end = 0  # Tracks furthest closing time in milliseconds
    next_interval_start_ts = 0

    # Initial interval calculation
    now = datetime.datetime.utcnow()
    current_second = now.second
    next_interval_second = ((current_second // 15) + 1) * 15
    if next_interval_second >= 60:
        next_interval_start = now.replace(second=0, microsecond=0) + datetime.timedelta(minutes=1)
    else:
        next_interval_start = now.replace(second=next_interval_second, microsecond=0)
    next_interval_start_ts = int(next_interval_start.timestamp() * 1000)
    print(f"Tracking {symbol.upper()} | Position Duration: {n*15}s | Thresholds: OCC={occ_threshold}%, OHC={ohc_threshold}%, OLC={olc_threshold}%")

    async with websockets.connect(BINANCE_WS_URL) as websocket:
        async for message in websocket:
            data = json.loads(message)
            ts = data["T"]

            if ts < next_interval_start_ts:
                continue

            price = float(data['p'])
            quantity = float(data['q'])

            interval_start = (ts // 15000) * 15000
            intervals.setdefault(interval_start, []).append({
                'timestamp': ts,
                'price': price,
                'quantity': quantity
            })

            # Process completed intervals
            to_process = [key for key in intervals.keys() if (key + 15000) <= ts]
            for key in sorted(to_process):
                trades = intervals.pop(key)
                if not trades:
                    continue

                open_price = trades[0]['price']
                close_price = trades[-1]['price']
                high_price = max(trade['price'] for trade in trades)
                low_price = min(trade['price'] for trade in trades)

                # Calculate metrics
                occ = (close_price - open_price) / open_price * 100
                ohc = (high_price - open_price) / open_price * 100
                olc = (low_price - open_price) / open_price * 100

                interval_start_dt = datetime.datetime.utcfromtimestamp(key / 1000)
                ohlc_time = interval_start_dt.strftime("%H:%M:%S UTC")

                # Build output
                output = [
                    f"\n📊 {symbol.upper()} {ohlc_time}",
                    f"Open: {open_price:.5f}",
                    f"High: {high_price:.5f} ({ohc:+.3f}%)",
                    f"Low: {low_price:.5f} ({olc:+.3f}%)",
                    f"Close: {close_price:.5f} ({occ:+.3f}%)"
                ]

                # Check long condition
                if occ >= occ_threshold and ohc >= ohc_threshold and olc >= olc_threshold:
                    output.append("🚨 LONG SIGNAL 🚨")
                    
                    # Calculate new end time
                    new_end = key + n * 15000  # n * 15s in milliseconds
                    if new_end > current_max_end:
                        current_max_end = new_end
                        end_time = datetime.datetime.utcfromtimestamp(new_end / 1000)
                        output.append(f"⏱ Max Close at {end_time.strftime('%H:%M:%S UTC')}")

                print("\n".join(output))

                # Check position closure
                if current_max_end != 0 and key == current_max_end:
                    print(f"💼 CLOSED POSITION | Exit at {ohlc_time} | Price: {close_price:.2f}")
                    current_max_end = 0  # Reset position tracking

async def main(symbol, occ_threshold=0.5, ohc_threshold=0.5, olc_threshold=0.5, n=4):
    await binance_ws(symbol, occ_threshold, ohc_threshold, olc_threshold, n)

# For Jupyter Notebook
symbol = "adausdt"
occ_threshold = 0.4  # Close% threshold
ohc_threshold = 0.0  # High% threshold
olc_threshold = -100  # Low% threshold
position_duration = 4  # Number of 15s intervals (4 = 60s total)
await main(symbol, occ_threshold, ohc_threshold, olc_threshold, position_duration)

# For .py script:
# if __name__ == "__main__":
#     symbol = "ethusdt"
#     asyncio.run(main(symbol, occ_threshold=0.2, ohc_threshold=0.3, olc_threshold=0.1, n=5))

In [ ]:
import asyncio
import websockets
import json
import datetime
import numpy as np

async def binance_ws(symbol, occ_threshold, ohc_threshold, olc_threshold, position_duration):
    """Fetch trade data with position duration management."""
    global intervals, next_interval_start_ts, current_max_end, long_signal_time, close_time

    BINANCE_WS_URL = f"wss://fstream.binance.com/ws/{symbol.lower()}@trade"
    intervals = {}
    current_max_end = 0  # Tracks furthest closing time in milliseconds
    next_interval_start_ts = 0
    long_signal_time = None  # Tracks when the long signal occurred
    close_time = None  # Tracks when to close the position

    # Initial interval calculation
    now = datetime.datetime.utcnow()
    current_second = now.second
    next_interval_second = ((current_second // 15) + 1) * 15
    if next_interval_second >= 60:
        next_interval_start = now.replace(second=0, microsecond=0) + datetime.timedelta(minutes=1)
    else:
        next_interval_start = now.replace(second=next_interval_second, microsecond=0)
    next_interval_start_ts = int(next_interval_start.timestamp() * 1000)
    print(f"Tracking {symbol.upper()} | Position Duration: {position_duration * 15}s | Thresholds: OCC={occ_threshold}%, OHC={ohc_threshold}%, OLC={olc_threshold}%")

    async with websockets.connect(BINANCE_WS_URL) as websocket:
        async for message in websocket:
            data = json.loads(message)
            ts = data["T"]

            if ts < next_interval_start_ts:
                continue

            price = float(data['p'])
            quantity = float(data['q'])

            interval_start = (ts // 15000) * 15000
            intervals.setdefault(interval_start, []).append({
                'timestamp': ts,
                'price': price,
                'quantity': quantity
            })

            # Process completed intervals
            to_process = [key for key in intervals.keys() if (key + 15000) <= ts]
            for key in sorted(to_process):
                trades = intervals.pop(key)
                if not trades:
                    continue

                open_price = trades[0]['price']
                close_price = trades[-1]['price']
                high_price = max(trade['price'] for trade in trades)
                low_price = min(trade['price'] for trade in trades if trade['price'] > 0) # Update the low_price calculation to filter out invalid prices

                # Calculate metrics
                occ = np.where(((close_price - open_price) == 0) | (open_price == 0), 0, (close_price - open_price) / open_price * 100)
                ohc = np.where(((high_price - open_price) == 0) | (open_price == 0), 0, (high_price - open_price) / open_price * 100)
                olc = np.where(((low_price - open_price) == 0) | (open_price == 0), 0, (low_price - open_price) / open_price * 100)


                interval_start_dt = datetime.datetime.utcfromtimestamp(key / 1000)
                ohlc_time = interval_start_dt.strftime("%H:%M:%S UTC")

                # Build output
                output = [
                    f"\n📊 {symbol.upper()} {ohlc_time}",
                    f"Open: {open_price:.5f}",
                    f"High: {high_price:.5f} ({ohc:+.3f}%)",
                    f"Low: {low_price:.5f} ({olc:+.3f}%)",
                    f"Close: {close_price:.5f} ({occ:+.3f}%)"
                ]

                # Check long condition
                if occ >= occ_threshold and ohc >= ohc_threshold and olc >= olc_threshold:
                    output.append("🚨 LONG SIGNAL 🚨")
                    long_signal_time = datetime.datetime.utcfromtimestamp(ts / 1000)  # Record long signal time
                    close_time = long_signal_time + datetime.timedelta(seconds=position_duration * 15)  # Set close time
                    print(f"⏱ Long signal detected at {long_signal_time.strftime('%H:%M:%S UTC')}")
                    print(f"⏱ Position will close at {close_time.strftime('%H:%M:%S UTC')}")

                print("\n".join(output))

                # Check if it's time to close the long position
                if close_time and datetime.datetime.utcnow() >= close_time:
                    print(f"💼 CLOSE LONG POSITION | Closed at {close_time.strftime('%H:%M:%S UTC')}")
                    close_time = None  # Reset close time
                    long_signal_time = None  # Reset long signal tracking

async def main(symbol, occ_threshold=0.5, ohc_threshold=0.5, olc_threshold=0.5, position_duration=4):
    await binance_ws(symbol, occ_threshold, ohc_threshold, olc_threshold, position_duration)

# For Jupyter Notebook
symbol = "adausdt"
occ_threshold = 0.4  # Close% threshold
ohc_threshold = 0.0  # High% threshold
olc_threshold = -100  # Low% threshold
position_duration = 4  # Position duration in 15s intervals (4 = 60s total)
await main(symbol, occ_threshold, ohc_threshold, olc_threshold, position_duration)

# For .py script:
# if __name__ == "__main__":
#     symbol = "ethusdt"
#     asyncio.run(main(symbol, occ_threshold=0.2, ohc_threshold=0.3, olc_threshold=0.1, position_duration=4))

/tmp/ipykernel_3139976/1428712482.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


Tracking ADAUSDT | Position Duration: 60s | Thresholds: OCC=0.4%, OHC=0.0%, OLC=-100%


/tmp/ipykernel_3139976/1428712482.py:65: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  interval_start_dt = datetime.datetime.utcfromtimestamp(key / 1000)



📊 ADAUSDT 06:26:15 UTC
Open: 0.92940
High: 0.92980 (+0.043%)
Low: 0.92930 (-0.011%)
Close: 0.92980 (+0.043%)

📊 ADAUSDT 06:26:30 UTC
Open: 0.92980
High: 0.93000 (+0.022%)
Low: 0.92910 (-0.075%)
Close: 0.92940 (-0.043%)

📊 ADAUSDT 06:26:45 UTC
Open: 0.92940
High: 0.92990 (+0.054%)
Low: 0.92940 (+0.000%)
Close: 0.92980 (+0.043%)

📊 ADAUSDT 06:27:00 UTC
Open: 0.92980
High: 0.93000 (+0.022%)
Low: 0.92910 (-0.075%)
Close: 0.92990 (+0.011%)

📊 ADAUSDT 06:27:15 UTC
Open: 0.93000
High: 0.93180 (+0.194%)
Low: 0.92970 (-0.032%)
Close: 0.93110 (+0.118%)

📊 ADAUSDT 06:27:30 UTC
Open: 0.93100
High: 0.93150 (+0.054%)
Low: 0.93080 (-0.021%)
Close: 0.93120 (+0.021%)

📊 ADAUSDT 06:27:45 UTC
Open: 0.93120
High: 0.93120 (+0.000%)
Low: 0.93020 (-0.107%)
Close: 0.93040 (-0.086%)

📊 ADAUSDT 06:28:00 UTC
Open: 0.93040
High: 0.93040 (+0.000%)
Low: 0.92960 (-0.086%)
Close: 0.92990 (-0.054%)

📊 ADAUSDT 06:28:15 UTC
Open: 0.92990
High: 0.93130 (+0.151%)
Low: 0.92980 (-0.011%)
Close: 0.93120 (+0.140%)

📊 ADAUSDT